<a href="https://colab.research.google.com/github/noushadkhan01/Projects/blob/master/Deep-Learning/ANN/Classification/ANN_model_for_classification_on_Chunk_dataset_for_bank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import time
start = time.time()
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('https://bit.ly/2QMuIPs')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.shape

(10000, 14)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [5]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


## check for null values

In [6]:
df.isnull().sum().sort_values(ascending = False)[:2]

Exited             0
EstimatedSalary    0
dtype: int64

**^^as u can see there is no null value**

In [7]:
df.skew()

RowNumber          0.000000
CustomerId         0.001149
CreditScore       -0.071607
Age                1.011320
Tenure             0.010991
Balance           -0.141109
NumOfProducts      0.745568
HasCrCard         -0.901812
IsActiveMember    -0.060437
EstimatedSalary    0.002085
Exited             1.471611
dtype: float64

In [8]:
np.log1p(df.Age).skew()

0.20336041516983316

## import important dependencies

In [0]:
import tensorflow as tf
from tensorflow import keras

In [10]:
df.head(1)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,101348.88,1


In [11]:
df = df.iloc[:, 3:]
df.head(1)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.0,1,1,1,101348.88,1


In [12]:
df = pd.get_dummies(df, drop_first = True)
df.head(1)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.0,1,1,1,101348.88,1,0,0,0


In [13]:
X = df.drop('Exited', 1)
y = df.Exited
X.shape, y.shape

((10000, 11), (10000,))

## StandardScaler

We need to scale out data before applying to neural network becuase of speed

In [0]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size = 0.3, random_state = 0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7000, 11), (3000, 11), (7000,), (3000,))

In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
sc = StandardScaler()

In [18]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train[:1], X_test[:1]

(array([[-0.09792126, -0.55759842, -1.03635146,  1.13249447,  0.81039385,
          0.64198477,  0.96683535, -0.76862426, -0.56987189, -0.5731713 ,
          0.92295821]]),
 array([[-0.55032881, -0.36597914,  1.0436811 ,  0.89172052, -0.92971564,
          0.64198477,  0.96683535,  1.61245299,  1.75478035, -0.5731713 ,
         -1.08347268]]))

In [19]:
y_train = y_train.values
y_test = y_test.values
y_train[:2], y_test[:2]

(array([1, 0]), array([0, 1]))

# Make ANN

In [0]:
from tensorflow import keras

## import Sequential from keras
**it is required to initialize our neural network model**

### Step 1:- create ann model by sequential

In [21]:
#1 first initialize our keras model by Sequential
model = keras.models.Sequential()
model

### Step 2:- Add dense layer

if a data is linenarly separable then their is no need to add hidden layer or don't even need a neural networks by convention

**Tip:-- ** no. of nodes in hidden layers is the average of no. of nodes in input layers and no. of node in output layer

In [22]:
ninput = 11 #no of independent variables
noutput = 1 #output is either 1 or either 0 so no of hidden layers in output is 1
nhidden = (ninput + noutput) / 2
nhidden

6.0

In [23]:
model.add(keras.layers.Dense(units = 6, activation = tf.nn.relu, input_dim = 11))

W0808 09:14:46.508322 140252812879744 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [24]:
model

### add 2nd layer

In [25]:
model.add(keras.layers.Dense(units = 6, activation = tf.nn.relu))
model

### Step 3:- add output layer

**there are two types of activation function for output of categorical varibles **
- 1**softmax**, which is used when dependent varible have more than two categorical variables
- 2 **sigmoid** , which is used when dependent varable have 2 categorical variables

In [0]:
model.add(keras.layers.Dense(units = 1, activation = tf.nn.sigmoid))

In [27]:
model

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


## Step 4:-- compile the model
apply stochastic gradient descent method

In [29]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model

W0808 09:14:46.735373 140252812879744 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Step 5:-- fit the model

batch_size = no. of observation afterwhich u want to update the weights

In [30]:
model.fit(X_train, y_train, batch_size = 10 , epochs = 70,verbose = 0)
end = time.time()
tpu_runtime = end - start
print(f'tpu run time is {tpu_runtime} seconds')

tpu run time is 52.2745897769928 seconds


### Step 7:- make predictions

In [47]:
model.evaluate(X_train, y_train, batch_size = 10)

7000/7000 [==============================] - 0s 64us/sample - loss: 0.3342 - acc: 0.8676


[0.33418220403737253, 0.8675714]

In [48]:
y_pred = model.predict(X_test)
y_pred[:2]

array([[0.3951859 ],
       [0.19935903]], dtype=float32)

In [0]:
y_pred = y_pred.reshape((y_pred.shape[0]))

In [50]:
y_pred

array([0.3951859 , 0.19935903, 0.12387514, ..., 0.06517959, 0.04615551,
       0.48346382], dtype=float32)

In [51]:
y_test

array([0, 1, 0, ..., 0, 0, 1])

In [52]:
y_pred.shape, y_test.shape

((3000,), (3000,))

In [53]:
from sklearn.metrics import confusion_matrix
f_ypred = [1 if i > .5 else 0 for i in y_pred]
confusion_matrix(y_test, f_ypred)

array([[2287,   92],
       [ 325,  296]])